In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import fiona
import pydash
import shapely.geometry
from shapely.geometry import Polygon, Point, shape
import pyproj
import numpy as np
from affine import Affine
from rasterio.features import rasterize

In [ ]:
from kk.dggs import DGGS
import kk.tools
from kk import uitools

dg = DGGS()

In [ ]:
poly, src_crs = kk.tools.load_polygons('/Users/kirill/wk/dggs/data/act_aea.shp')
poly = poly[0]
display(poly, src_crs)

In [ ]:
scale_level = 12#13

overlaps = dg.compute_overlap(scale_level, *poly.boundary.xy, crs=src_crs)

if len(overlaps) != 1:
    print('Warning polygon crosses top-level DGGS regions, this is not supported yet')
    

roi = overlaps[0]
display( roi )
roi = roi.align_by(5)
display(roi)

tr, *_ = dg.pixel_coord_transform(roi, native=True, no_offset=True)


poly_m = shapely.ops.transform(dg.to_native(src_crs), poly)
display(poly_m)

poly_pix = shapely.ops.transform(lambda x,y: tr.inverse(x,y), poly_m)
display(poly_pix)

[int(b) for b in poly_pix.bounds]

In [ ]:
im = rasterize([poly_pix], out_shape=roi.shape)

ff = dg.mk_display_helper()

fig = plt.figure(figsize=(6,6))
ax = fig.add_axes((0,0,1,1))


im, extent = ff(roi.addr, im)

ax.imshow(im, extent=extent)

uitools.plot_roi(roi, 'y-', ax=ax, alpha=0.5, linewidth=2)
uitools.plot_roi(roi.scale_up(3), 'b-', ax=ax, alpha=0.1, linewidth=2)
uitools.plot_roi(roi.scale_up(4), 'b-', ax=ax, alpha=0.1, linewidth=2)
uitools.plot_roi(roi.scale_up(5), 'b-', ax=ax, alpha=0.1, linewidth=2)




In [ ]:
roi, roi.scale_up(1), roi.scale_up(5), roi.align_by(5)